In [ ]:
#Import Dependencies
from citipy import citipy
from config import gkey
from pprint import pprint
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
import json

In [2]:
#Randomly generate latitude and longitude numbers. lookup nearest city using citipy

# Empty series to hold all the names of cities and countries
random_City = []
random_Country = []
loopcnt = 0

#Loop through until at least 500 unique city names are found
while len(random_City) < 500 :
    # Loop counter to track number of loops made
    loopcnt +=1
    
    # Randomly generate latitude
    lat_pos = random.randint(0,2)
    base_lat = random.randint(0,91)
    dec_lat = random.random()/100
    lat = base_lat + dec_lat
    if lat_pos == 1:
        lat = lat * -1
    #print(f"The lattitude is: {lat}") 
    
    # Randomly generate longitude
    lng_pos = random.randint(0,2)
    base_lng = random.randint(0,181)
    dec_lng = random.random()/100
    lng = base_lng + dec_lng
    if lng_pos == 1:
        lng = lng * -1
    #print(f"The longitude is: {lng}") 
    
    # Call citipy file to get nearest city based on random lat and lng
    city = citipy.nearest_city(lat, lng)
    
    # Test to see if the random city is already in the list, if not, add it to the city and country lists
    foundCity = False
    for j in range(len(random_City)):
        if city.city_name == random_City[j]:
            foundCity = True
    if foundCity==False  :   
        random_City.append(city.city_name)
        random_Country.append(city.country_code.upper())

In [3]:
print(random_City + random_Country)

['nanao', 'batagay-alyta', 'dikson', 'tiksi', 'severo-kurilsk', 'luderitz', 'qaanaaq', 'barentsburg', 'pevek', 'vaini', 'cidreira', 'port elizabeth', 'changli', 'belushya guba', 'fare', 'butaritari', 'busselton', 'ribeira grande', 'vardo', 'port macquarie', 'kapaa', 'yar-sale', 'san jose', 'tailai', 'rikitea', 'kaitangata', 'grindavik', 'tubruq', 'dudinka', 'khatanga', 'port alfred', 'vostok', 'tuktoyaktuk', 'east london', 'savonlinna', 'kango', 'tabulbah', 'san andres', 'yellowknife', 'iskateley', 'kodiak', 'komsomolskiy', 'hobart', 'bousso', 'ushuaia', 'kirchzarten', 'ayan', 'hasaki', 'gilgit', 'ventspils', 'kushiro', 'nanortalik', 'louisbourg', 'lompoc', 'oktyabrskiy', 'alta floresta', 'payo', 'talnakh', 'bengkulu', 'constitucion', 'zanzibar', 'grand-santi', 'jamestown', 'buncrana', 'ciocanesti', 'illoqqortoormiut', 'attawapiskat', 'shimoda', 'nizhneyansk', 'sentyabrskiy', 'salalah', 'bonavista', 'nemuro', 'bandarbeyla', 'albany', 'shirak', 'mokrousovo', 'ishigaki', 'novikovo', 'tsi

In [4]:

# Build query Url
base_url = "http://api.openweathermap.org/data/2.5/weather?"
api_key = gkey
units = "imperial"

query_url = base_url + "appid=" + api_key + "&units=" + units + "&q="
print(query_url)

http://api.openweathermap.org/data/2.5/weather?appid=7e63dfd3aecf1c96c595c91b157ea726&units=imperial&q=


In [6]:

# Loop through the list of cities and perform request for data on each
search_data = []
city_List = []
country_List = []

searchCnt = 0
print("Begin API Call to OpenWeather Database ")
print("---------------------------------------")
for city in random_City:
    searchCnt += 1
    print(f"Processing record {searchCnt} for the city: {city}")   
    response = requests.get(query_url + city).json()
    searchCityID = response.get("id")
    if response.get("id"):
        print(f"     Record found for city: {city}    city id: {searchCityID}")
        search_data.append(response)
        city_List.append(city)
        country_List.append(random_Country[searchCnt-1])
    else:
        print(f"     No weather record found for city: {city}")


Begin API Call to OpenWeather Database 
---------------------------------------
Processing record 1 for the city: nanao
     Record found for city: nanao    city id: 1855670
Processing record 2 for the city: batagay-alyta
     Record found for city: batagay-alyta    city id: 2027042
Processing record 3 for the city: dikson
     Record found for city: dikson    city id: 1507390
Processing record 4 for the city: tiksi
     Record found for city: tiksi    city id: 2015306
Processing record 5 for the city: severo-kurilsk
     Record found for city: severo-kurilsk    city id: 2121385
Processing record 6 for the city: luderitz
     Record found for city: luderitz    city id: 3355672
Processing record 7 for the city: qaanaaq
     Record found for city: qaanaaq    city id: 3831208
Processing record 8 for the city: barentsburg
     No weather record found for city: barentsburg
Processing record 9 for the city: pevek
     Record found for city: pevek    city id: 2122090
Processing record 10 for 

     Record found for city: guerrero negro    city id: 4021858
Processing record 83 for the city: klaksvik
     Record found for city: klaksvik    city id: 2618795
Processing record 84 for the city: bairiki
     No weather record found for city: bairiki
Processing record 85 for the city: hithadhoo
     Record found for city: hithadhoo    city id: 1282256
Processing record 86 for the city: nikki
     Record found for city: nikki    city id: 2392505
Processing record 87 for the city: nyurba
     Record found for city: nyurba    city id: 2018735
Processing record 88 for the city: bluff
     Record found for city: bluff    city id: 2175403
Processing record 89 for the city: kamenskoye
     No weather record found for city: kamenskoye
Processing record 90 for the city: bolshoye sorokino
     No weather record found for city: bolshoye sorokino
Processing record 91 for the city: fortuna
     Record found for city: fortuna    city id: 2517679
Processing record 92 for the city: geraldton
     R

     Record found for city: shalakusha    city id: 495983
Processing record 164 for the city: shawnee
     Record found for city: shawnee    city id: 4551177
Processing record 165 for the city: krasnoselkup
     No weather record found for city: krasnoselkup
Processing record 166 for the city: norman wells
     Record found for city: norman wells    city id: 6089245
Processing record 167 for the city: oranjemund
     Record found for city: oranjemund    city id: 3354071
Processing record 168 for the city: bethel
     Record found for city: bethel    city id: 5880568
Processing record 169 for the city: ahipara
     Record found for city: ahipara    city id: 2194098
Processing record 170 for the city: zhigalovo
     Record found for city: zhigalovo    city id: 2012532
Processing record 171 for the city: warqla
     No weather record found for city: warqla
Processing record 172 for the city: asyut
     Record found for city: asyut    city id: 359783
Processing record 173 for the city: las

     Record found for city: progreso    city id: 3440963
Processing record 246 for the city: saldanha
     Record found for city: saldanha    city id: 2737599
Processing record 247 for the city: kieta
     Record found for city: kieta    city id: 2094027
Processing record 248 for the city: langxiang
     Record found for city: langxiang    city id: 2036237
Processing record 249 for the city: saint-leu
     Record found for city: saint-leu    city id: 6690297
Processing record 250 for the city: antalaha
     Record found for city: antalaha    city id: 1071296
Processing record 251 for the city: avarua
     Record found for city: avarua    city id: 4035715
Processing record 252 for the city: leningradskiy
     Record found for city: leningradskiy    city id: 2123814
Processing record 253 for the city: dunedin
     Record found for city: dunedin    city id: 2191562
Processing record 254 for the city: barela
     Record found for city: barela    city id: 1277008
Processing record 255 for t

     Record found for city: yerbogachen    city id: 2012956
Processing record 327 for the city: livingston
     Record found for city: livingston    city id: 4073383
Processing record 328 for the city: karauzyak
     No weather record found for city: karauzyak
Processing record 329 for the city: maniitsoq
     Record found for city: maniitsoq    city id: 3421982
Processing record 330 for the city: nandu
     Record found for city: nandu    city id: 6957978
Processing record 331 for the city: bukachacha
     Record found for city: bukachacha    city id: 2026023
Processing record 332 for the city: gravdal
     Record found for city: gravdal    city id: 3147822
Processing record 333 for the city: rovnoye
     Record found for city: rovnoye    city id: 501091
Processing record 334 for the city: ixtapa
     Record found for city: ixtapa    city id: 4004293
Processing record 335 for the city: victoria
     Record found for city: victoria    city id: 1733782
Processing record 336 for the city

     Record found for city: straumen    city id: 3135728
Processing record 408 for the city: maceio
     Record found for city: maceio    city id: 3395981
Processing record 409 for the city: martapura
     Record found for city: martapura    city id: 1636022
Processing record 410 for the city: dondo
     Record found for city: dondo    city id: 1024696
Processing record 411 for the city: tumannyy
     No weather record found for city: tumannyy
Processing record 412 for the city: vizinga
     Record found for city: vizinga    city id: 473263
Processing record 413 for the city: pervomayskiy
     Record found for city: pervomayskiy    city id: 510911
Processing record 414 for the city: jingdezhen
     Record found for city: jingdezhen    city id: 1805680
Processing record 415 for the city: cayenne
     Record found for city: cayenne    city id: 3382160
Processing record 416 for the city: winslow
     Record found for city: winslow    city id: 2633808
Processing record 417 for the city: de

     Record found for city: eldorado    city id: 4007812
Processing record 490 for the city: manuk mangkaw
     Record found for city: manuk mangkaw    city id: 1701394
Processing record 491 for the city: laredo
     Record found for city: laredo    city id: 4705349
Processing record 492 for the city: agirish
     Record found for city: agirish    city id: 1512101
Processing record 493 for the city: aden
     Record found for city: aden    city id: 415189
Processing record 494 for the city: la paz
     Record found for city: la paz    city id: 4000900
Processing record 495 for the city: nantucket
     Record found for city: nantucket    city id: 4944903
Processing record 496 for the city: mys shmidta
     No weather record found for city: mys shmidta
Processing record 497 for the city: mandalgovi
     Record found for city: mandalgovi    city id: 2030065
Processing record 498 for the city: roswell
     Record found for city: roswell    city id: 5488441
Processing record 499 for the cit